In [34]:
#pip install numba
#pip install numpy
#pip install pandas
#pip install scipy

In [4]:
from numba import njit
import numpy as np

@njit(cache=True, fastmath=True)
def search():
    # Preallocate space for up to 1 million solutions, each storing 20 integer values.
    max_results = 1_000_000
    results = np.empty((max_results, 20), dtype=np.int64)

    # Define the integer search range [-3, -2, -1, 0, 1, 2, 3].
    r = np.arange(-3, 4, dtype=np.int64)
    squares = np.array([x * x for x in r], dtype=np.int64)

    count = 0  # Counter for valid results found.

    # Enumerate all integer combinations of (x, y, z, w, k, l, m, n, f, g, h, i, a, b, c, d)
    # Each nested loop corresponds to one variable taking values from r.
    for xi in range(len(r)):
        x, x2 = r[xi], squares[xi]
        for yi in range(len(r)):
            y, y2 = r[yi], squares[yi]
            for zi in range(len(r)):
                z, z2 = r[zi], squares[zi]
                for wi in range(len(r)):
                    w, w2 = r[wi], squares[wi]
                    for ki in range(len(r)):
                        k, k2 = r[ki], squares[ki]
                        for li in range(len(r)):
                            l, l2 = r[li], squares[li]
                            for mi in range(len(r)):
                                m, m2 = r[mi], squares[mi]
                                for ni in range(len(r)):
                                    n, n2 = r[ni], squares[ni]

                                    # Constraint 1: linear condition linking x, y, z, w with k, l, m, n.
                                    if 2 * k * x + k + 2 * l * y + y + m * z + n * w != -1:
                                        continue

                                    for fi in range(len(r)):
                                        f, f2 = r[fi], squares[fi]
                                        for gi in range(len(r)):
                                            g, g2 = r[gi], squares[gi]
                                            for hi in range(len(r)):
                                                h, h2 = r[hi], squares[hi]
                                                for ii in range(len(r)):
                                                    i, i2 = r[ii], squares[ii]

                                                    # Constraint 2: another linear relation involving (f, g, h, i).
                                                    if 2 * f * k + 2 * g * l + g + h * m + i * n != -1:
                                                        continue

                                                    # Constraint 3: coupling between (f, g, h, i) and (x, y, z, w).
                                                    if 2 * f * x + f + 2 * g * y + h * z + i * w != 0:
                                                        continue

                                                    for ai in range(len(r)):
                                                        a, a2 = r[ai], squares[ai]
                                                        for bi in range(len(r)):
                                                            b, b2 = r[bi], squares[bi]
                                                            for ci in range(len(r)):
                                                                c, c2 = r[ci], squares[ci]
                                                                for di in range(len(r)):
                                                                    d, d2 = r[di], squares[di]

                                                                    # Three additional constraints coupling (a, b, c, d)
                                                                    # with each of the previous variable groups.
                                                                    if 2 * a * x + a + 2 * b * y + c * z + d * w != -1:
                                                                        continue
                                                                    if 2 * a * k + 2 * b * l + b + c * m + d * n != 0:
                                                                        continue
                                                                    if 2 * a * f + 2 * b * g + c * h + d * i != 0:
                                                                        continue

                                                                    # Derived quadratic quantities (parameterized expressions).
                                                                    p_val = 1 + 2 * x2 + 2 * x + 2 * y2 + z2 + w2
                                                                    q_val = 2 * k2 + 2 * l2 + 2 * l + 1 + m2 + n2
                                                                    r_val = 2 * a2 + 2 * b2 + c2 + d2 
                                                                    s_val = 2 * f2 + 2 * g2 + h2 + i2

                                                                    # Derived denominators and boundaries.
                                                                    denom = -3 - 2 * p_val - 2 * q_val + 4 * p_val * q_val
                                                                    if denom == 0:
                                                                        continue

                                                                    # Critical thresholds based on p and q.
                                                                    critical_r = (-2 + 4 * p_val) / denom
                                                                    boundary1 = (767 + 512 * p_val) / (-512 + 1024 * p_val)
                                                                    boundary2 = (3 + 2 * p_val) / (-2 + 4 * p_val)

                                                                    # Secondary threshold for s in terms of p, q, r.
                                                                    num_s = 4092 - 2 * r_val + 4 * q_val * r_val
                                                                    denom_s = (
                                                                        2 - 4 * p_val - 3 * r_val
                                                                        - 2 * p_val * r_val - 2 * q_val * r_val
                                                                        + 4 * p_val * q_val * r_val
                                                                    )
                                                                    if denom_s == 0:
                                                                        continue
                                                                    thresh_s = num_s / denom_s

                                                                    # Evaluate logical conditions defining "validity".
                                                                    valid = False

                                                                    if p_val < 1:
                                                                        valid = False
                                                                    else:
                                                                        # Piecewise validity checks based on boundaries.
                                                                        if q_val <= boundary1:
                                                                            if r_val < critical_r:
                                                                                valid = s_val <= thresh_s
                                                                            elif r_val == critical_r:
                                                                                valid = True
                                                                            else:
                                                                                valid = s_val >= thresh_s
                                                                        elif boundary1 < q_val < boundary2:
                                                                            if r_val < critical_r:
                                                                                valid = s_val <= thresh_s
                                                                            else:
                                                                                valid = s_val >= thresh_s
                                                                        elif q_val == boundary2:
                                                                            # Special boundary handling.
                                                                            top = 4092 - 2 * r_val + (4 * boundary2 * r_val)
                                                                            bottom = (
                                                                                2 - 4 * p_val - 3 * r_val
                                                                                - 2 * p_val * r_val - (2 * boundary2 * r_val)
                                                                                + (4 * p_val * boundary2 * r_val)
                                                                            )
                                                                            if bottom != 0:
                                                                                valid = s_val >= top / bottom
                                                                        elif q_val > boundary2:
                                                                            if r_val < critical_r:
                                                                                valid = s_val >= thresh_s
                                                                            elif r_val == critical_r:
                                                                                valid = True
                                                                            else:
                                                                                valid = s_val <= thresh_s

                                                                    # Record solution if all conditions pass.
                                                                    if valid and count < max_results:
                                                                        results[count, :] = (
                                                                            x, y, z, w, a, b, c, d,
                                                                            k, l, m, n, f, g, h, i,
                                                                            p_val, q_val, r_val, s_val
                                                                        )
                                                                        count += 1

    # Return only the filled portion of the array.
    return results[:count]

# Run search and print how many valid solutions were found.
valid_solutions = search()
print(f"Total valid solutions: {len(valid_solutions)}")

Total valid solutions: 42496


In [5]:
import numpy as np

def filter_unique_pqrs(solutions):
    # Create a set to store unique (p, q, r, s) tuples we've already seen.
    # A Python set is used here for O(1) average-time membership checks.
    seen_pqrs = set()

    # This list will collect only the first occurrence of each unique (p, q, r, s).
    filtered = []

    # Iterate through each row of the solutions array.
    # Each 'row' is a 1D NumPy array of length 20, corresponding to one valid solution.
    for row in solutions:
        # Extract columns 16–19, which store (p_val, q_val, r_val, s_val).
        # Convert the slice to a tuple so it can be hashed and stored in a set.
        pqrs = tuple(row[16:20])

        # If this (p, q, r, s) combination hasn't been encountered yet, keep it.
        if pqrs not in seen_pqrs:
            seen_pqrs.add(pqrs)   # Mark this combination as seen.
            filtered.append(row)  # Retain the full solution row.

    # Convert the filtered list of unique rows back into a NumPy array,
    # preserving the original integer dtype of the input.
    return np.array(filtered, dtype=solutions.dtype)

# Apply the uniqueness filter to the full solution set.
unique_pqrs_solutions = filter_unique_pqrs(valid_solutions)

# Print how many distinct (p, q, r, s) combinations were found.
print(len(unique_pqrs_solutions))

1206


In [7]:
import numpy as np

def build_basis(sol):
    # Extract parameters (x,y,z,w), (a,b,c,d), (k,l,m,n), (f,g,h,i)
    if isinstance(sol, dict):
        x, y, z, w = sol['x'], sol['y'], sol['z'], sol['w']
        a, b, c, d = sol['a'], sol['b'], sol['c'], sol['d']
        k, l, m, n = sol['k'], sol['l'], sol['m'], sol['n']
        f, g, h, i = sol['f'], sol['g'], sol['h'], sol['i']
    else:
        x, y, z, w = sol[0], sol[1], sol[2], sol[3]
        a, b, c, d = sol[4], sol[5], sol[6], sol[7]
        k, l, m, n = sol[8], sol[9], sol[10], sol[11]
        f, g, h, i = sol[12], sol[13], sol[14], sol[15]

    # Basis vectors of the associated 6-dimensional lattice
    v1 = np.array([a, a, b, b, c, d])
    v2 = np.array([1, -1, 0, 0, 0, 0])
    v3 = np.array([x, x + 1, y, y, z, w])
    v4 = np.array([k, k, l, l + 1, m, n])
    v5 = np.array([f, f, g, g, h, i])
    v6 = np.array([0, 0, 1, -1, 0, 0])

    return np.array([v1, v2, v3, v4, v5, v6], dtype=np.float32)

In [8]:
import numpy as np

def wu_obstruction(basis):
    # Wu vector k = sum of basis vectors in Z^6
    W = np.sum(basis, axis=0)
    k = W
    k_squared_sum = np.sum(k**2)

    # Indices where k is odd; corresponds to R₀ ⊂ {1,…,6}
    Ro = [i for i, ki in enumerate(k) if ki % 2 == 1]

    # Wu-type inequality: k·k > 4n − 3|R₀| with n = dim = len(k)
    lhs = k_squared_sum
    rhs = 4 * len(k) - 3 * len(Ro)
    return lhs > rhs

def remove_wu_obstructed(solutions):
    # Discard solutions whose associated basis violates the Wu inequality
    filtered = []
    for sol in solutions:
        basis = build_basis(sol)
        if not wu_obstruction(basis):
            filtered.append(sol)
    return filtered

# Apply Wu obstruction filter to the unique (p,q,r,s) solutions
# wu_obstructed_solutions = remove_wu_obstructed(valid_solutions)
wu_obstructed_solutions = remove_wu_obstructed(unique_pqrs_solutions)

# Unpack parameters (for inspection / further constraints)
for sol in wu_obstructed_solutions:
    if isinstance(sol, dict):
        x, y, z, w = sol['x'], sol['y'], sol['z'], sol['w']
        a, b, c, d = sol['a'], sol['b'], sol['c'], sol['d']
        k, l, m, n = sol['k'], sol['l'], sol['m'], sol['n']
        f, g, h, i = sol['f'], sol['g'], sol['h'], sol['i']
    else:
        x, y, z, w = sol[0], sol[1], sol[2], sol[3]
        a, b, c, d = sol[4], sol[5], sol[6], sol[7]
        k, l, m, n = sol[8], sol[9], sol[10], sol[11]
        f, g, h, i = sol[12], sol[13], sol[14], sol[15]

len(wu_obstructed_solutions)

410

In [9]:
import numpy as np

def is_full_rank(basis, tol=1e-10):
    """
    Checks if the lattice basis matrix is of full row rank.
    A full-rank basis (rank = n) ensures the corresponding lattice 
    spans the full ambient dimension.
    """
    return np.linalg.matrix_rank(basis, tol=tol) == basis.shape[0]


def filter_full_rank_lattices(solutions):
    """
    Keeps only those solutions whose constructed basis is full rank.
    This ensures that degenerate (lower-dimensional) lattices are excluded.
    """
    return [sol for sol in solutions if is_full_rank(build_basis(sol))]


def removed_full_rank(solutions):
    """
    Returns the complement — solutions whose bases fail to be full rank.
    """
    return [sol for sol in solutions if not is_full_rank(build_basis(sol))]


# Evaluate and report lattice rank statistics after Wu obstruction filtering
fullRankFiltered = filter_full_rank_lattices(wu_obstructed_solutions)
notFullRank = removed_full_rank(wu_obstructed_solutions)

print(f"{len(fullRankFiltered)} out of {len(wu_obstructed_solutions)} are full-rank.")
print(f"{len(notFullRank)} are not full-rank.")

395 out of 410 are full-rank.
15 are not full-rank.


In [10]:
def reformat_to_dict(solutions):
    """
    Converts each solution row (length 20) into a labeled dictionary.

    Each key corresponds to the ordered variables:
        (x, y, z, w, a, b, c, d, k, l, m, n, f, g, h, i, p, q, r, s)

    This format is convenient for downstream symbolic or algebraic use
    (e.g., for printing, exporting, or constructing explicit lattice data).
    """
    keys = ["x", "y", "z", "w", "a", "b", "c", "d",
            "k", "l", "m", "n", "f", "g", "h", "i",
            "p", "q", "r", "s"]
    out = []
    for row in solutions:
        # Pair each key with the corresponding value and cast to int for clarity
        out.append({k: int(v) for k, v in zip(keys, row)})
    return out  # Return a list of dictionaries

# Produce a human-readable list of solutions after filtering
pretty_solutions = reformat_to_dict(fullRankFiltered)

In [11]:
def canonical_pqrs(p, q, r, s):
    # Canonicalize (p, q, r, s) under H-plumbing symmetries:
    # interchange of (p,q) and/or (r,s) preserves equivalence.
    forms = [
        (p, q, r, s),
        (q, p, r, s),
        (p, q, s, r),
        (q, p, s, r),
    ]

    # Add degenerate symmetric representatives when r = s or p = q.
    if r == s:
        forms.append((q, p, r, r))
    if p == q:
        forms.append((p, p, s, r))

    return min(forms)


def filter_symmetric_solutions(solutions):
    # Remove duplicates modulo H-plumbing symmetry in (p,q,r,s)
    seen = set()
    unique = []
    for sol in solutions:
        p, q, r, s = sol['p'], sol['q'], sol['r'], sol['s']
        key = canonical_pqrs(p, q, r, s)
        if key not in seen:
            seen.add(key)
            unique.append(sol)
    return unique


# Identify distinct representatives up to the canonical symmetry
unique_pretty_solutions = filter_symmetric_solutions(pretty_solutions)
print(f"Unique solutions (modulo symmetry) {len(unique_pretty_solutions)}, out of {len(pretty_solutions)}")

Unique solutions (modulo symmetry) 187, out of 395


In [12]:
#better tolerance checker
from numba import njit
import numpy as np

@njit
def generate_unit_cube(dim):
    num_corners = 1 << dim
# Better numerical tolerance checker for cubiquity
from numba import njit
import numpy as np

@njit
def generate_unit_cube(dim):
    """
    Generate all 2^dim vertices of the unit cube [0,1]^dim.
    """
    num_corners = 1 << dim
    cube = np.zeros((num_corners, dim), dtype=np.float32)
    for i in range(num_corners):
        for j in range(dim):
            cube[i, j] = (i >> j) & 1
    return cube

@njit
def is_cubiquitous_fast(basis, bound=3):
    """
    Numerical cubiquity test: for each translated unit cube
    in [-bound, bound]^dim, check that at least one cube vertex
    is within numerical tolerance of the lattice spanned by 'basis'.
    """
    dim = basis.shape[1]
    basis_T = basis.T
    unit_cube = generate_unit_cube(dim)

    # Approximate condition number and scale of basis_T to set a robust tolerance
    u, s, vh = np.linalg.svd(basis_T)
    cond = s[0] / s[-1] if s[-1] > 0 else 1e10
    scale = max(np.linalg.norm(basis_T), 1.0)
    tol = cond * np.finfo(np.float32).eps * scale * 10  # safety factor

    cube_range = range(-bound, bound + 1)
    for i0 in cube_range:
        for i1 in cube_range:
            for i2 in cube_range:
                for i3 in cube_range:
                    for i4 in cube_range:
                        for i5 in cube_range:
                            coord = np.array([i0, i1, i2, i3, i4, i5], dtype=np.float32)
                            cube_passed = False
                            # Require at least one vertex of each translated cube
                            # to be numerically realized as a lattice point
                            for corner in unit_cube:
                                point = corner + coord
                                coeffs = np.linalg.solve(basis_T, point)
                                rounded = np.round(coeffs)
                                recon = basis_T @ rounded
                                if np.all(np.abs(recon - point) < tol):
                                    cube_passed = True
                                    break
                            if not cube_passed:
                                return False
    return True


cubiquitous_solutions = []

for sol in unique_pretty_solutions:
    basis = build_basis(sol).astype(np.float32)
    if is_cubiquitous_fast(basis, bound=2):
        cubiquitous_solutions.append(sol)
        #print("Cubiquitous:", sol)

print("Number of cubiquitous solutions:", len(cubiquitous_solutions))

Number of cubiquitous solutions: 64


Cubiqutious: {'x': -1, 'y': -1, 'z': 0, 'w': 0, 'a': -1, 'b': 1, 'c': 1, 'd': 0, 'k': 0, 'l': 0, 'm': -1, 'n': -1, 'f': 0, 'g': 0, 'h': 0, 'i': 1, 'p': 3, 'q': 3, 'r': 5, 's': 1}
Cubiqutious: {'x': -1, 'y': -1, 'z': 0, 'w': 0, 'a': 1, 'b': 0, 'c': 0, 'd': 0, 'k': 0, 'l': 0, 'm': -1, 'n': -1, 'f': 0, 'g': 0, 'h': 0, 'i': 1, 'p': 3, 'q': 3, 'r': 2, 's': 1}
Cubiqutious: {'x': -1, 'y': -1, 'z': 0, 'w': 0, 'a': 1, 'b': 0, 'c': 0, 'd': 0, 'k': 0, 'l': 0, 'm': -1, 'n': 0, 'f': 0, 'g': 0, 'h': 1, 'i': -2, 'p': 3, 'q': 2, 'r': 2, 's': 5}
Cubiqutious: {'x': -1, 'y': -1, 'z': 0, 'w': 0, 'a': -1, 'b': 1, 'c': 1, 'd': 1, 'k': 0, 'l': 0, 'm': -1, 'n': 0, 'f': 0, 'g': 0, 'h': 1, 'i': -1, 'p': 3, 'q': 2, 'r': 6, 's': 2}
Cubiqutious: {'x': -1, 'y': -1, 'z': 0, 'w': 0, 'a': 1, 'b': 0, 'c': 0, 'd': 0, 'k': 0, 'l': 0, 'm': -1, 'n': 0, 'f': 0, 'g': 0, 'h': 1, 'i': -1, 'p': 3, 'q': 2, 'r': 2, 's': 2}
Cubiqutious: {'x': -1, 'y': -1, 'z': 0, 'w': 0, 'a': 1, 'b': 0, 'c': 0, 'd': -2, 'k': 0, 'l': 0, 'm': -1, 'n

In [13]:
def canonical_pqrs(p, q, r, s):
    """
    Canonical representative of (p, q, r, s) under the basic H-plumbing symmetry:
    (p, q, r, s) ~ (q, p, s, r), with extra degenerate cases when p = q or r = s.
    """
    forms = [
        (p, q, r, s),
        (q, p, s, r),
    ]
    if r == s:
        forms.append((q, p, r, r))
    if p == q:
        forms.append((p, p, s, r))
    return min(forms)

def filter_symmetric_and_small_lattices(solutions):
    """
    Mod out solutions by the above symmetry in (p, q, r, s), keeping
    one canonical representative per orbit.
    """
    seen = set()
    unique = []

    for sol in solutions:
        p, q, r, s = sol['p'], sol['q'], sol['r'], sol['s']

        # Optional size cutoff for (p, q, r, s); currently disabled.
        if r < 2 or s < 2:
            continue

        key = canonical_pqrs(p, q, r, s)
        if key not in seen:
            seen.add(key)
            unique.append(sol)

    return unique

filtered = filter_symmetric_and_small_lattices(cubiquitous_solutions)
print(f"Filtered unique solutions: {len(filtered)}")
for sol in filtered:
    print(sol)

Filtered unique solutions: 39
{'x': -2, 'y': -1, 'z': -1, 'w': 0, 'a': 0, 'b': 0, 'c': 1, 'd': -1, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'f': 1, 'g': -1, 'h': -1, 'i': -1, 'p': 8, 'q': 1, 'r': 2, 's': 6}
{'x': -2, 'y': 0, 'z': -1, 'w': 0, 'a': 0, 'b': 0, 'c': 1, 'd': 1, 'k': 0, 'l': -1, 'm': 1, 'n': -1, 'f': 0, 'g': 1, 'h': 0, 'i': 0, 'p': 6, 'q': 3, 'r': 2, 's': 2}
{'x': -1, 'y': -1, 'z': -2, 'w': -2, 'a': -1, 'b': 0, 'c': 1, 'd': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'f': 0, 'g': -1, 'h': 0, 'i': 1, 'p': 11, 'q': 1, 'r': 3, 's': 3}
{'x': -1, 'y': -1, 'z': -2, 'w': -2, 'a': 1, 'b': 0, 'c': 0, 'd': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'f': 0, 'g': -1, 'h': 0, 'i': 1, 'p': 11, 'q': 1, 'r': 2, 's': 3}
{'x': -1, 'y': -1, 'z': -2, 'w': -1, 'a': -1, 'b': 0, 'c': 1, 'd': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'f': 0, 'g': -1, 'h': 0, 'i': 2, 'p': 8, 'q': 1, 'r': 3, 's': 6}
{'x': -1, 'y': -1, 'z': -2, 'w': -1, 'a': 1, 'b': 0, 'c': 0, 'd': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'f': 0, 'g': -1, 'h': 1, 'i': 0, '

In [14]:
import numpy as np

def enumerate_lattices(solutions, decimals=6):
    """
    Enumerate all lattices from the given solutions.
    For each sol:
      - Construct its 6×6 basis matrix B with rows v₁,…,v₆.
      - Record det(B) > 0 (absolute value of determinant).
      - Compute squared norms and assign:
            p = ||v₃||², q = ||v₄||²,
            r = ||v₁||², s = ||v₅||²,
            t = ||v₂||², u = ||v₆||².
    """
    data = []
    for sol in solutions:
        B = build_basis(sol).astype(np.float64)
        det_val = abs(np.linalg.det(B))  # ensure positivity
        det = float(np.round(det_val, decimals))
        norms_sq = np.sum(B**2, axis=1)
        norms_sq = np.round(norms_sq, decimals)

        v1_sq, v2_sq, v3_sq, v4_sq, v5_sq, v6_sq = norms_sq
        p, q, r, s, t, u = float(v3_sq), float(v4_sq), float(v1_sq), float(v5_sq), float(v2_sq), float(v6_sq)

        data.append({
            "solution": sol,
            "basis": np.round(B, decimals),
            "det": det,
            "p": p, "q": q, "r": r, "s": s, "t": t, "u": u
        })
    return data


# Example usage
lattice_data = enumerate_lattices(filtered, decimals=6)

for idx, L in enumerate(lattice_data):
    print(f"--- Lattice {idx} ---")
    print("parameters:", L["solution"])
    print("basis matrix (rows v1..v6):\n", L["basis"])
    print("det(B):", L["det"])
    print("p,q,r,s,t,u =", L["p"], L["q"], L["r"], L["s"], L["t"], L["u"])
    print()

--- Lattice 0 ---
parameters: {'x': -2, 'y': -1, 'z': -1, 'w': 0, 'a': 0, 'b': 0, 'c': 1, 'd': -1, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'f': 1, 'g': -1, 'h': -1, 'i': -1, 'p': 8, 'q': 1, 'r': 2, 's': 6}
basis matrix (rows v1..v6):
 [[ 0.  0.  0.  0.  1. -1.]
 [ 1. -1.  0.  0.  0.  0.]
 [-2. -1. -1. -1. -1.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 1.  1. -1. -1. -1. -1.]
 [ 0.  0.  1. -1.  0.  0.]]
det(B): 8.0
p,q,r,s,t,u = 8.0 1.0 2.0 6.0 2.0 2.0

--- Lattice 1 ---
parameters: {'x': -2, 'y': 0, 'z': -1, 'w': 0, 'a': 0, 'b': 0, 'c': 1, 'd': 1, 'k': 0, 'l': -1, 'm': 1, 'n': -1, 'f': 0, 'g': 1, 'h': 0, 'i': 0, 'p': 6, 'q': 3, 'r': 2, 's': 2}
basis matrix (rows v1..v6):
 [[ 0.  0.  0.  0.  1.  1.]
 [ 1. -1.  0.  0.  0.  0.]
 [-2. -1.  0.  0. -1.  0.]
 [ 0.  0. -1.  0.  1. -1.]
 [ 0.  0.  1.  1.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.]]
det(B): 12.0
p,q,r,s,t,u = 6.0 3.0 2.0 2.0 2.0 2.0

--- Lattice 2 ---
parameters: {'x': -1, 'y': -1, 'z': -2, 'w': -2, 'a': -1, 'b': 0, 'c': 1, 'd': 0, 'k': 0, 'l': 0, 'm':

In [52]:
from itertools import product  # (not used below, but kept if needed elsewhere)

def constraint_expr(p, q, r, s, t, u):
    """
    Polynomial constraint in the six parameters (p, q, r, s, t, u),
    corresponding to the lattice inequality under consideration.
    """
    return (
        r * t + s * t
        - p * r * s * t
        + r * u
        - p * r * s * u
        + s * u
        - q * r * t * u
        - q * s * t * u
        - r * s * t * u
        + p * q * r * s * t * u
    )

def generate_equiv_classes(p, q, r, s, t, u):
    """
    Symmetry in (p, q, r, s, t, u) induced by swapping
    (p, q) ↔ (q, p) and simultaneously (r, s, t, u) ↔ (s, r, u, t).
    """
    return [
        (p, q, r, s, t, u),
        (q, p, s, r, u, t),
        (p, q, t, u, r, s),
        (q, p, u, t, s, r),
    ]

def get_canonical_signature(p, q, r, s, t, u):
    """
    Choose a canonical representative for the symmetry class
    by taking the lexicographically minimal tuple.
    """
    return min(generate_equiv_classes(p, q, r, s, t, u))


# Parameter ranges
p_range = range(3, 25)
q_range = range(3, 25)
r_range = range(2, 24)
s_range = range(2, 24)
t_range = range(2, 24)
u_range = range(2, 24)

canonical_signatures = set()
valid_tuples = []

for p in p_range:
    for q in q_range:
        for r in r_range:
            for s in s_range:
                for t in t_range:
                    for u in u_range:

                        total = p + q + r + s + t + u
                        # Global bound on the sum of parameters
                        if total > 34:
                            continue

                        # Additional upper bounds depending on size (currently disabled)
                        # if all(x >= 2 for x in (p, q, r, s, t, u)):
                        #     if total > 19 and constraint_expr(p, q, r, s, t, u) > 1024:
                        #         continue
                        # if all(x >= 3 for x in (p, q, r, s, t, u)):
                        #     if total > 9 and constraint_expr(p, q, r, s, t, u) > 81:
                        #         continue

                        expr_val = constraint_expr(p, q, r, s, t, u)

                        # Record tuples with constraint value in the desired range
                        if 0 < expr_val <= 4096:
                            canon = get_canonical_signature(p, q, r, s, t, u)
                            if canon not in canonical_signatures:
                                canonical_signatures.add(canon)
                                valid_tuples.append((p, q, r, s, t, u, expr_val))

# Example of how to inspect the results (kept commented for batch runs)
# for p, q, r, s, t, u, expr_val in valid_tuples:
#     print(
#         f"(p={p}, q={q}, r={r}, s={s}, t={t}, u={u}) "
#         f"-> expr = {expr_val}, sum = {p+q+r+s+t+u}"
#     )

In [ ]:
#We need to add the Wu-obstruction justification for only considering weights up to 24 and 25

In [54]:
'''
# Assuming `valid_tuples` is the list from earlier with 7 elements per tuple (p, q, r, s, t, u, expr)
if not valid_tuples:
    print("No valid tuples found.")
else:
    max_p = max(t[0] for t in valid_tuples)
    max_q = max(t[1] for t in valid_tuples)
    max_r = max(t[2] for t in valid_tuples)
    max_s = max(t[3] for t in valid_tuples)
    max_t = max(t[4] for t in valid_tuples)
    max_u = max(t[5] for t in valid_tuples)

    print(f"Maximum values seen across valid tuples:")
    print(f"p: {max_p}, q: {max_q}, r: {max_r}, s: {max_s}, t: {max_t}, u: {max_u}")
'''

'\n# Assuming `valid_tuples` is the list from earlier with 7 elements per tuple (p, q, r, s, t, u, expr)\nif not valid_tuples:\n    print("No valid tuples found.")\nelse:\n    max_p = max(t[0] for t in valid_tuples)\n    max_q = max(t[1] for t in valid_tuples)\n    max_r = max(t[2] for t in valid_tuples)\n    max_s = max(t[3] for t in valid_tuples)\n    max_t = max(t[4] for t in valid_tuples)\n    max_u = max(t[5] for t in valid_tuples)\n\n    print(f"Maximum values seen across valid tuples:")\n    print(f"p: {max_p}, q: {max_q}, r: {max_r}, s: {max_s}, t: {max_t}, u: {max_u}")\n    '

In [54]:
import math

def remove_special_2x2_blocks(valid_tuples):
    """
    Remove tuples where the 2×2 block parameters (r,s) or (t,u) are (2,2).
    Tuples are of the form (p, q, r, s, t, u, expr_val).
    """
    return [
        tup for tup in valid_tuples
        if not (tup[2] == 2 and tup[3] == 2)    # (r, s) != (2,2)
        and not (tup[4] == 2 and tup[5] == 2)  # (t, u) != (2,2)
    ]

filtered_tuples = remove_special_2x2_blocks(valid_tuples)

def is_perfect_square(n: int) -> bool:
    """
    Test whether n is a perfect square.
    """
    root = math.isqrt(n)
    return root * root == n

def filter_perfect_square_determinants(tuples):
    """
    Keep only those tuples whose determinant (here expr_val = tup[6])
    is a perfect square.
    """
    return [tup for tup in tuples if is_perfect_square(tup[6])]

square_determinant_tuples = filter_perfect_square_determinants(filtered_tuples)

def remove_special_2x2_blocks2(seq):
    """
    Further exclude tuples whose (r,s) or (t,u) equal (2,3) or (3,2).
    """
    bad = {(2, 3), (3, 2)}
    return [
        t for t in seq
        if (t[2], t[3]) not in bad   # (r, s) not in bad
        and (t[4], t[5]) not in bad  # (t, u) not in bad
    ]

filtered_tuples2 = remove_special_2x2_blocks2(square_determinant_tuples)

In [1]:
#filtered_tuples2 #where is 3,5,3,2,3,2?
#first we ran this code with the (3,2) blocks, and that's how we found the additional cubiquitous case.
#afterwards, we removed all of those lattices, in order to shorten the list

In [56]:
# Core search routine (placed before the tuple generator)
from numba import njit
import numpy as np

@njit(cache=True, fastmath=True)
def search():
    # Store up to 10^6 integral solutions; each row has 21 entries.
    max_results = 1_000_000
    results = np.empty((max_results, 21), dtype=np.int64)

    # Search range and precomputed squares
    r = np.arange(-3, 4, dtype=np.int64)
    squares = np.array([x * x for x in r], dtype=np.int64)
    count = 0

    for ai in range(len(r)):
        a, a2 = r[ai], squares[ai]
        for bi in range(len(r)):
            b, b2 = r[bi], squares[bi]
            for ci in range(len(r)):
                c, c2 = r[ci], squares[ci]
                for xi in range(len(r)):
                    x, x2 = r[xi], squares[xi]
                    for yi in range(len(r)):
                        y, y2 = r[yi], squares[yi]
                        for zi in range(len(r)):
                            z, z2 = r[zi], squares[zi]
                            for wi in range(len(r)):
                                w, w2 = r[wi], squares[wi]

                                # First linear constraint in (a,b,c,x,y,z,w)
                                if a * y + b * z + c * w != -1:
                                    continue

                                for fi in range(len(r)):
                                    f, f2 = r[fi], squares[fi]
                                    for gi in range(len(r)):
                                        g, g2 = r[gi], squares[gi]
                                        for hi in range(len(r)):
                                            h, h2 = r[hi], squares[hi]
                                            for ii in range(len(r)):
                                                i, i2 = r[ii], squares[ii]

                                                # Orthogonality-type constraint in (a,b,c,f,g,h,i)
                                                if a * g + b * h + c * i != 0:
                                                    continue

                                                # Mixed constraint coupling (x,y,z,w) and (f,g,h,i)
                                                if (
                                                    6 * x * f + 3 * f + 2 * x
                                                    + g * y + z * h + w * i
                                                ) != -2:
                                                    continue

                                                for ki in range(len(r)):
                                                    k, k2 = r[ki], squares[ki]
                                                    for li in range(len(r)):
                                                        l, l2 = r[li], squares[li]
                                                        for mi in range(len(r)):
                                                            m, m2 = r[mi], squares[mi]
                                                            for ni in range(len(r)):
                                                                n, n2 = r[ni], squares[ni]

                                                                # Further linear constraints in (a,b,c,k,l,m,n)
                                                                if a * l + b * m + c * n != 0:
                                                                    continue
                                                                if (
                                                                    6 * x * k + 3 * k
                                                                    + y * l + z * m + w * n
                                                                ) != 0:
                                                                    continue
                                                                if (
                                                                    6 * f * k + 2 * k
                                                                    + g * l + h * m + i * n
                                                                ) != -1:
                                                                    continue

                                                                # Quadratic quantities p,q,r,s corresponding
                                                                # to the norms of the relevant blocks
                                                                p_val = 6 * x2 + 6 * x + 2 + y2 + z2 + w2
                                                                q_val = 6 * f2 + 4 * f + 1 + g2 + h2 + i2
                                                                r_val = a2 + b2 + c2
                                                                s_val = 6 * k2 + l2 + m2 + n2

                                                                # Global inequality bound (≤ 4096)
                                                                expr = (
                                                                    6
                                                                    + 3 * r_val
                                                                    - 6 * p_val * r_val
                                                                    + 2 * s_val
                                                                    - 6 * q_val * s_val
                                                                    - 5 * r_val * s_val
                                                                    - 2 * p_val * r_val * s_val
                                                                    - 3 * q_val * r_val * s_val
                                                                    + 6 * p_val * q_val * r_val * s_val
                                                                )

                                                                if expr > 4096:
                                                                    continue

                                                                if count < max_results:
                                                                    # Row format:
                                                                    # [1, a, b, c, 0, x, y, z, w,
                                                                    #  f, g, h, i, k, l, m, n,
                                                                    #  p, q, r, s]
                                                                    results[count, :] = (
                                                                        1, a, b, c, 0, x, y, z, w,
                                                                        f, g, h, i, k, l, m, n,
                                                                        p_val, q_val, r_val, s_val
                                                                    )
                                                                    count += 1

    return results[:count]


# Run and report
valid_solutions1 = search()
print(f"Total valid solutions: {len(valid_solutions1)}")

Total valid solutions: 6312


In [55]:
from numba import njit
import numpy as np

@njit(cache=True, fastmath=True)
def search():
    # Up to 10^6 integral solutions; each row has 21 entries
    max_results = 1_000_000
    results = np.empty((max_results, 21), dtype=np.int64)

    # Search range {-3,…,3} and precomputed squares
    r = np.arange(-3, 4, dtype=np.int64)
    squares = np.array([x * x for x in r], dtype=np.int64)
    count = 0

    for ai in range(len(r)):
        a, a2 = r[ai], squares[ai]
        for bi in range(len(r)):
            b, b2 = r[bi], squares[bi]
            for ci in range(len(r)):
                c, c2 = r[ci], squares[ci]
                for di in range(len(r)):
                    d, d2 = r[di], squares[di]
                    for xi in range(len(r)):
                        x, x2 = r[xi], squares[xi]
                        for yi in range(len(r)):
                            y, y2 = r[yi], squares[yi]
                            for zi in range(len(r)):
                                z, z2 = r[zi], squares[zi]
                                for wi in range(len(r)):
                                    w, w2 = r[wi], squares[wi]

                                    # First linear compatibility condition
                                    if (
                                        2 * a * x + a
                                        + 2 * b * y
                                        + 2 * c * z
                                        + y * c + b * z
                                        + d * w
                                    ) != -1:
                                        continue

                                    for fi in range(len(r)):
                                        f, f2 = r[fi], squares[fi]
                                        for gi in range(len(r)):
                                            g, g2 = r[gi], squares[gi]
                                            for hi in range(len(r)):
                                                h, h2 = r[hi], squares[hi]
                                                for ii in range(len(r)):
                                                    i, i2 = r[ii], squares[ii]

                                                    # Mixed condition between (a,b,c,d) and (f,g,h,i)
                                                    if (
                                                        2 * a * f
                                                        + 2 * b * g
                                                        + 2 * c * h
                                                        + b * h + c * g
                                                        + b + c
                                                        + d * i
                                                    ) != 0:
                                                        continue

                                                    # Mixed condition between (x,y,z,w) and (f,g,h,i)
                                                    if (
                                                        2 * x * f + f
                                                        + 2 * y * g
                                                        + 2 * z * h
                                                        + y * h + z * g
                                                        + y + z
                                                        + w * i
                                                    ) != -1:
                                                        continue 

                                                    for ki in range(len(r)):
                                                        k, k2 = r[ki], squares[ki]
                                                        for li in range(len(r)):
                                                            l, l2 = r[li], squares[li]
                                                            for mi in range(len(r)):
                                                                m, m2 = r[mi], squares[mi]
                                                                for ni in range(len(r)):
                                                                    n, n2 = r[ni], squares[ni]                            

                                                                    # Conditions coupling the three blocks
                                                                    if (
                                                                        2 * a * k
                                                                        + 2 * b * l
                                                                        + 2 * c * m
                                                                        + b * m + c * l
                                                                        + d * n
                                                                    ) != 0:
                                                                        continue
                                                                    if (
                                                                        2 * x * k + k
                                                                        + 2 * y * l
                                                                        + 2 * z * m
                                                                        + y * m + z * l
                                                                        + w * n
                                                                    ) != 0:
                                                                        continue
                                                                    if (
                                                                        2 * f * k
                                                                        + 2 * g * l
                                                                        + 2 * h * m
                                                                        + g * m + h * l
                                                                        + l + m
                                                                        + i * n
                                                                    ) != -1:
                                                                        continue

                                                                    # Quadratic parameters p,q,r,s from the norms
                                                                    p_val = (
                                                                        2 * x2 + 2 * x + 1
                                                                        + 2 * y2
                                                                        + 2 * z2
                                                                        + 2 * z * y
                                                                        + w2
                                                                    )
                                                                    q_val = (
                                                                        2 * f2
                                                                        + 2 * g2
                                                                        + 2 * h2
                                                                        + 2 * g * h
                                                                        + 2 * g + 2 * h
                                                                        + i2 + 1
                                                                    )
                                                                    r_val = 2 * a2 + 2 * b2 + 2 * c2 + 2 * b * c + d2
                                                                    s_val = 2 * k2 + 2 * l2 + 2 * m2 + 2 * l * m + n2

                                                                    # Global inequality (determinant-type bound ≤ 4096)
                                                                    expr = (
                                                                        6
                                                                        + 3 * r_val
                                                                        - 6 * p_val * r_val
                                                                        + 2 * s_val
                                                                        - 6 * q_val * s_val
                                                                        - 5 * r_val * s_val
                                                                        - 2 * p_val * r_val * s_val
                                                                        - 3 * q_val * r_val * s_val
                                                                        + 6 * p_val * q_val * r_val * s_val
                                                                    )

                                                                    if expr > 4096:
                                                                        continue

                                                                    if count < max_results:
                                                                        # Row format:
                                                                        # [2, a,b,c,d, x,y,z,w, f,g,h,i, k,l,m,n, p,q,r,s]
                                                                        results[count, :] = (
                                                                            2,
                                                                            a, b, c, d,
                                                                            x, y, z, w, 
                                                                            f, g, h, i,
                                                                            k, l, m, n, 
                                                                            p_val, q_val, r_val, s_val
                                                                        )
                                                                        count += 1

    return results[:count]

# Run and report
valid_solutions2 = search()
print(f"Total valid solutions: {len(valid_solutions2)}")

Total valid solutions: 17088


In [57]:
valid_solutions = np.vstack((valid_solutions1, valid_solutions2))

In [59]:
import numpy as np

def filter_unique_pqrs(solutions):
    seen_pqrs = set()
    filtered = []

    for sol in solutions:
        # Columns 17–20 correspond to (p, q, r, s)
        pqrs = (sol[17], sol[18], sol[19], sol[20])
        if pqrs not in seen_pqrs:
            seen_pqrs.add(pqrs)
            filtered.append(sol)

    return np.array(filtered, dtype=solutions.dtype)

unique_pqrs_solutions = filter_unique_pqrs(valid_solutions)
print(len(unique_pqrs_solutions))

809


In [60]:
import numpy as np

def build_basis2(sol):
    """
    Given a solution sol (either an array row or a dict),
    return the 6×6 lattice basis with rows v₁,…,v₆.
    The shape of the basis depends on the source type (1 or 2).
    """
    # Extract parameters and source tag
    if isinstance(sol, dict):
        source = sol['source']
        x, y, z, w = sol['x'], sol['y'], sol['z'], sol['w']
        a, b, c, d = sol['a'], sol['b'], sol['c'], sol['d']
        k, l, m, n = sol['k'], sol['l'], sol['m'], sol['n']
        f, g, h, i = sol['f'], sol['g'], sol['h'], sol['i']
    else:
        source = sol[0]
        a, b, c, d = sol[1], sol[2], sol[3], sol[4]
        x, y, z, w = sol[5], sol[6], sol[7], sol[8]
        f, g, h, i = sol[9], sol[10], sol[11], sol[12]
        k, l, m, n = sol[13], sol[14], sol[15], sol[16]
        
    if source == 1:
        # Basis type 1
        v1 = np.array([1, -1, 0, 0, 0, 0], dtype=np.int32)
        v2 = np.array([0, 0, 0, a, b, c], dtype=np.int32)
        v3 = np.array([x, x + 1, 2 * x + 1, y, z, w], dtype=np.int32)
        v4 = np.array([f, f, 2 * f + 1, g, h, i], dtype=np.int32)
        v5 = np.array([1, 1, -1, 0, 0, 0], dtype=np.int32)
        v6 = np.array([k, k, 2 * k, l, m, n], dtype=np.int32)

    elif source == 2:
        # Basis type 2
        v1 = np.array([1, -1, 0, 0, 0, 0], dtype=np.int32)
        v2 = np.array([a, a, b, c, b + c, d], dtype=np.int32)
        v3 = np.array([x, x + 1, y, z, y + z, w], dtype=np.int32)
        v4 = np.array([f, f, g, h, g + h + 1, i], dtype=np.int32)
        v5 = np.array([0, 0, 1, 1, -1, 0], dtype=np.int32)
        v6 = np.array([k, k, l, m, l + m, n], dtype=np.int32)
        
    else:
        raise ValueError(f"Invalid source value: {source}")
        
    return np.array([v1, v2, v3, v4, v5, v6], dtype=np.int32)

In [61]:
def remove_wu_obstructed2(solutions):
    filtered = []
    for sol in solutions:
        basis = build_basis2(sol)
        if not wu_obstruction(basis):
            filtered.append(sol)
    return filtered

wu_obstructed_solutions2 = remove_wu_obstructed2(unique_pqr_solutions)
print(len(wu_obstructed_solutions2))

236


In [62]:
def filter_full_rank_lattices2(solutions):
    """
    Keep only those solutions whose lattice basis (via build_basis2)
    has full rank.
    """
    return [sol for sol in solutions if is_full_rank(build_basis2(sol))]

def removed_full_rank2(solutions):
    """
    Return solutions whose bases are not full rank.
    """
    return [sol for sol in solutions if not is_full_rank(build_basis2(sol))]

# Apply rank filter after Wu obstruction removal
fullRankFiltered2 = filter_full_rank_lattices2(wu_obstructed_solutions2)
notFullRank2 = removed_full_rank2(wu_obstructed_solutions2)

print(f"{len(fullRankFiltered2)} out of {len(wu_obstructed_solutions2)} are full-rank.")
print(f"{len(notFullRank2)} are not full-rank.")

228 out of 236 are full-rank.
8 are not full-rank.


In [63]:
def reformat_to_dict(solutions):
    """
    Convert each solution row (length 21) into a labeled dictionary.
    The first entry is the source identifier (1 or 2),
    followed by parameters (a,b,c,d), (x,y,z,w), (f,g,h,i), (k,l,m,n),
    and the derived quantities (p,q,r,s).
    """
    formatted = []
    for sol in solutions:
        formatted.append({
            'source': sol[0],   # lattice family tag: 1 or 2
            'a': sol[1], 'b': sol[2], 'c': sol[3], 'd': sol[4],
            'x': sol[5], 'y': sol[6], 'z': sol[7], 'w': sol[8],
            'f': sol[9], 'g': sol[10], 'h': sol[11], 'i': sol[12],
            'k': sol[13], 'l': sol[14], 'm': sol[15], 'n': sol[16],
            'p': sol[17], 'q': sol[18], 'r': sol[19], 's': sol[20]
        })
    return formatted


# Reformat the full-rank, Wu-unobstructed solutions into dictionaries
pretty_solutions = reformat_to_dict(fullRankFiltered2)

# Note: for source == 1, all embeddings satisfy d == 0 by construction.

In [64]:
cubiquitous_solutions = []

for sol in pretty_solutions:
    basis = build_basis2(sol).astype(np.float32)
    if is_cubiquitous_fast(basis, bound=2):
        cubiquitous_solutions.append(sol)
        #print("Cubiqutious:", sol)

print("Number of cubiquitous solutions:", len(cubiquitous_solutions))

Number of cubiquitous solutions: 81


In [1]:
from numba import njit
import numpy as np
import time

@njit(cache=True, fastmath=True)
def solve_system(max_results=1_000_000_000):
    """
    Integer search for 5-component vectors (a,b,c,d,f) with given bilinear
    relations and squared norms (p,q,r,t,u).

    Output shape: (N, 31)
      [source=1, a(5), b(5), c(5), d(5), f(5), p, q, r, t, u]
    """
    r = np.arange(-3, 4, dtype=np.int64)
    results = np.empty((max_results, 31), dtype=np.int64)
    count = 0

    def in_range(x):
        return -3 <= x <= 3

    for a1 in r:
        for a2 in r:
            for a3 in r:
                for a4 in r:
                    for a5 in r:
                        for b2 in r:
                            for b3 in r:
                                for b4 in r:
                                    for b5 in r:
                                        sab = a2*b2 + a3*b3 + a4*b4 + a5*b5

                                        if a1 != 0:
                                            rhs = -1 - a1 - sab
                                            denom = 2*a1
                                            if rhs % denom != 0:
                                                continue
                                            b1 = rhs // denom
                                            if not in_range(b1):
                                                continue
                                        else:
                                            if sab != -1:
                                                continue

                                        def do_after_b1(a1,a2,a3,a4,a5,b1,b2,b3,b4,b5):
                                            nonlocal count
                                            for c2 in r:
                                                for c3 in r:
                                                    for c4 in r:
                                                        for c5 in r:
                                                            sac = a2*c2 + a3*c3 + a4*c4 + a5*c5
                                                            sbc = b2*c2 + b3*c3 + b4*c4 + b5*c5
                                                            if a1 != 0:
                                                                rhs = -sac
                                                                denom = 2*a1
                                                                if rhs % denom != 0:
                                                                    continue
                                                                c1 = rhs // denom
                                                                if not in_range(c1):
                                                                    continue
                                                            else:
                                                                if sac != 0:
                                                                    continue
                                                                rhs = -1 - sbc
                                                                denom = 2*b1 + 1
                                                                if rhs % denom != 0:
                                                                    continue
                                                                c1 = rhs // denom
                                                                if not in_range(c1):
                                                                    continue
                                                            if (2*b1 + 1)*c1 + sbc != -1:
                                                                continue
                                                            for d2 in r:
                                                                for d3 in r:
                                                                    for d4 in r:
                                                                        for d5 in r:
                                                                            sad = a2*d2 + a3*d3 + a4*d4 + a5*d5
                                                                            sbd = b2*d2 + b3*d3 + b4*d4 + b5*d5
                                                                            scd = c2*d2 + c3*d3 + c4*d4 + c5*d5
                                                                            if a1 != 0:
                                                                                rhs = -sad
                                                                                denom = 2*a1
                                                                                if rhs % denom != 0:
                                                                                    continue
                                                                                d1 = rhs // denom
                                                                                if not in_range(d1):
                                                                                    continue
                                                                            else:
                                                                                if sad != 0:
                                                                                    continue
                                                                                rhs = -sbd
                                                                                denom = 2*b1 + 1
                                                                                if rhs % denom != 0:
                                                                                    continue
                                                                                d1 = rhs // denom
                                                                                if not in_range(d1):
                                                                                    continue
                                                                            if (2*b1 + 1)*d1 + sbd != 0:
                                                                                continue
                                                                            if 2*c1*d1 + scd != -1:
                                                                                continue
                                                                            for f2 in r:
                                                                                for f3 in r:
                                                                                    for f4 in r:
                                                                                        for f5 in r:
                                                                                            saf = a2*f2 + a3*f3 + a4*f4 + a5*f5
                                                                                            sbf = b2*f2 + b3*f3 + b4*f4 + b5*f5
                                                                                            scf = c2*f2 + c3*f3 + c4*f4 + c5*f5
                                                                                            sdf = d2*f2 + d3*f3 + d4*f4 + d5*f5
                                                                                            if a1 != 0:
                                                                                                rhs = -saf
                                                                                                denom = 2*a1
                                                                                                if rhs % denom != 0:
                                                                                                    continue
                                                                                                f1 = rhs // denom
                                                                                                if not in_range(f1):
                                                                                                    continue
                                                                                            else:
                                                                                                if saf != 0:
                                                                                                    continue
                                                                                                rhs = -sbf
                                                                                                denom = 2*b1 + 1
                                                                                                if rhs % denom != 0:
                                                                                                    continue
                                                                                                f1 = rhs // denom
                                                                                                if not in_range(f1):
                                                                                                    continue
                                                                                            if (2*b1 + 1)*f1 + sbf != 0:
                                                                                                continue
                                                                                            if 2*c1*f1 + scf != -1:
                                                                                                continue
                                                                                            if 2*d1*f1 + sdf != 0:
                                                                                                continue
                                                                                            t_val = 2*a1*a1 + a2*a2 + a3*a3 + a4*a4 + a5*a5
                                                                                            p_val = 2*b1*b1 + 2*b1 + 1 + b2*b2 + b3*b3 + b4*b4 + b5*b5
                                                                                            q_val = 2*c1*c1 + c2*c2 + c3*c3 + c4*c4 + c5*c5
                                                                                            r_val = 2*d1*d1 + d2*d2 + d3*d3 + d4*d4 + d5*d5
                                                                                            u_val = 2*f1*f1 + f2*f2 + f3*f3 + f4*f4 + f5*f5
                                                                                            if count < max_results:
                                                                                                results[count, :] = np.array([
                                                                                                    1,
                                                                                                    a1,a2,a3,a4,a5,
                                                                                                    b1,b2,b3,b4,b5,
                                                                                                    c1,c2,c3,c4,c5,
                                                                                                    d1,d2,d3,d4,d5,
                                                                                                    f1,f2,f3,f4,f5,
                                                                                                    p_val,q_val,r_val,t_val,u_val
                                                                                                ], dtype=np.int64)
                                                                                                count += 1
                                        if a1 != 0:
                                            do_after_b1(a1,a2,a3,a4,a5,b1,b2,b3,b4,b5)
                                        else:
                                            for b1_tmp in r:
                                                do_after_b1(a1,a2,a3,a4,a5,int(b1_tmp),b2,b3,b4,b5)

    return results[:count]


# ---- Time the full solve ----
start_time = time.perf_counter()
sols1 = solve_system(max_results=100_000_000_000)
end_time = time.perf_counter()

print("Family 1 solutions:", len(sols1))
print(f"Elapsed time: {end_time - start_time:.3f} seconds")

Family 1 solutions: 22975872
Elapsed time: 2149.723 seconds


In [2]:
from itertools import product
import numpy as np
import os, math, time

def dot_param(u, off_u, v, off_v):
    """
    Inner product in R^6 for
      (u0, u1, u0+u1+off_u, u2, u3, u4)
      (v0, v1, v0+v1+off_v, v2, v3, v4).
    """
    return (u[0]*v[0] + u[1]*v[1]
            + (u[0]+u[1]+off_u)*(v[0]+v[1]+off_v)
            + u[2]*v[2] + u[3]*v[3] + u[4]*v[4])

def norm2_param(u, off_u):
    """Squared norm under the same parametrized embedding."""
    return (u[0]*u[0] + u[1]*u[1] + (u[0]+u[1]+off_u)**2
            + u[2]*u[2] + u[3]*u[3] + u[4]*u[4])

def inrng(x, lo=-3, hi=3):
    return lo <= x <= hi

def solve_2x2_Z(a11, a12, a21, a22, b1, b2):
    """
    Solve:
        a11 x + a12 y = b1
        a21 x + a22 y = b2
    over Z. Returns (x, y, ok, singular).
    """
    det = a11*a22 - a12*a21
    if det == 0:
        return 0, 0, False, True
    n1 =  b1*a22 - a12*b2
    n2 =  a11*b2 - b1*a21
    if (n1 % det) or (n2 % det):
        return 0, 0, False, False
    return n1//det, n2//det, True, False

def dump_embeddings_to_file(
    path,
    domain=range(-3,4),
    chunk_rows=200_000,
    overwrite=True
):
    """
    Enumerate all 5-tuples a,b,c,d,f ∈ domain^5 satisfying the dot_param
    relations and write them in chunks to a binary file as int16 rows of
    length 30:
        [a1..a5, b1..b5, c1..c5, d1..d5, f1..f5, p, q, r, t, u]
    with p,q,r,t,u the squared norms of b,c,d,a,f.
    """
    if overwrite and os.path.exists(path):
        os.remove(path)

    buf = np.empty((chunk_rows, 30), dtype=np.int16)
    w = 0
    total = 0

    with open(path, "ab", buffering=1024*1024) as f:
        def flush():
            nonlocal w, total
            if w:
                buf[:w].tofile(f)
                total += w
                w = 0

        for a1, a2, a3, a4, a5 in product(domain, repeat=5):
            a = [a1, a2, a3, a4, a5]
            coef_b1 = 2*a1 + a2

            for b2, b3, b4, b5 in product(domain, repeat=4):
                tail = ((a1 + 2*a2)*b2 +
                        (a1 + a2) +
                        a3*b3 + a4*b4 + a5*b5)

                if coef_b1 != 0:
                    rhs = -1 - tail
                    if rhs % coef_b1:
                        continue
                    b1 = rhs // coef_b1
                    if not inrng(b1):
                        continue
                    b = [b1, b2, b3, b4, b5]

                    A11, A12 = 2*a[0] + a[1], a[0] + 2*a[1]
                    B11, B12 = 2*b[0] + b[1] + 1, b[0] + 2*b[1] + 1

                    for c3, c4, c5 in product(domain, repeat=3):
                        rhs1 = -(a[2]*c3 + a[3]*c4 + a[4]*c5)
                        rhs2 = -1 - (b[2]*c3 + b[3]*c4 + b[4]*c5)
                        c1, c2, ok, sing = solve_2x2_Z(A11, A12, B11, B12, rhs1, rhs2)
                        if sing or not ok or not (inrng(c1) and inrng(c2)):
                            continue
                        c = [c1, c2, c3, c4, c5]
                        if dot_param(a, 0, c, 0) != 0 or dot_param(b, 1, c, 0) != -1:
                            continue

                        for d3, d4, d5 in product(domain, repeat=3):
                            rhs1d = -(a[2]*d3 + a[3]*d4 + a[4]*d5)
                            rhs2d = -(b[2]*d3 + b[3]*d4 + b[4]*d5)
                            d1, d2, okd, singd = solve_2x2_Z(A11, A12, B11, B12, rhs1d, rhs2d)
                            if singd or not okd or not (inrng(d1) and inrng(d2)):
                                continue
                            d = [d1, d2, d3, d4, d5]
                            if dot_param(a, 0, d, 0) != 0 or dot_param(b, 1, d, 0) != 0:
                                continue
                            if dot_param(c, 0, d, 0) != -1:
                                continue

                            for f3, f4, f5 in product(domain, repeat=3):
                                rhs1f = -(a[2]*f3 + a[3]*f4 + a[4]*f5)
                                rhs2f = -(b[2]*f3 + b[3]*f4 + b[4]*f5)
                                f1, f2, okf, singf = solve_2x2_Z(A11, A12, B11, B12, rhs1f, rhs2f)
                                if singf or not okf or not (inrng(f1) and inrng(f2)):
                                    continue
                                fvec = [f1, f2, f3, f4, f5]
                                if (dot_param(a, 0, fvec, 0) != 0 or
                                    dot_param(b, 1, fvec, 0) != 0 or
                                    dot_param(c, 0, fvec, 0) != -1 or
                                    dot_param(d, 0, fvec, 0) != 0):
                                    continue

                                t_val = norm2_param(a, 0)
                                p_val = norm2_param(b, 1)
                                q_val = norm2_param(c, 0)
                                r_val = norm2_param(d, 0)
                                u_val = norm2_param(fvec, 0)

                                row = (a + b + c + d + fvec +
                                       [p_val, q_val, r_val, t_val, u_val])
                                buf[w, :] = np.array(row, dtype=np.int16)
                                w += 1
                                if w == chunk_rows:
                                    flush()
                else:
                    if tail != -1:
                        continue
                    for b1 in domain:
                        b = [b1, b2, b3, b4, b5]
                        A11, A12 = 2*a[0] + a[1], a[0] + 2*a[1]
                        B11, B12 = 2*b[0] + b[1] + 1, b[0] + 2*b[1] + 1

                        for c3, c4, c5 in product(domain, repeat=3):
                            rhs1 = -(a[2]*c3 + a[3]*c4 + a[4]*c5)
                            rhs2 = -1 - (b[2]*c3 + b[3]*c4 + b[4]*c5)
                            c1, c2, ok, sing = solve_2x2_Z(A11, A12, B11, B12, rhs1, rhs2)
                            if sing or not ok or not (inrng(c1) and inrng(c2)):
                                continue
                            c = [c1, c2, c3, c4, c5]
                            if dot_param(a, 0, c, 0) != 0 or dot_param(b, 1, c, 0) != -1:
                                continue

                            for d3, d4, d5 in product(domain, repeat=3):
                                rhs1d = -(a[2]*d3 + a[3]*d4 + a[4]*d5)
                                rhs2d = -(b[2]*d3 + b[3]*d4 + b[4]*d5)
                                d1, d2, okd, singd = solve_2x2_Z(A11, A12, B11, B12, rhs1d, rhs2d)
                                if singd or not okd or not (inrng(d1) and inrng(d2)):
                                    continue
                                d = [d1, d2, d3, d4, d5]
                                if dot_param(a, 0, d, 0) != 0 or dot_param(b, 1, d, 0) != 0:
                                    continue
                                if dot_param(c, 0, d, 0) != -1:
                                    continue

                                for f3, f4, f5 in product(domain, repeat=3):
                                    rhs1f = -(a[2]*f3 + a[3]*f4 + a[4]*f5)
                                    rhs2f = -(b[2]*f3 + b[3]*f4 + b[4]*f5)
                                    f1, f2, okf, singf = solve_2x2_Z(A11, A12, B11, B12, rhs1f, rhs2f)
                                    if singf or not okf or not (inrng(f1) and inrng(f2)):
                                        continue
                                    fvec = [f1, f2, f3, f4, f5]
                                    if (dot_param(a, 0, fvec, 0) != 0 or
                                        dot_param(b, 1, fvec, 0) != 0 or
                                        dot_param(c, 0, fvec, 0) != -1 or
                                        dot_param(d, 0, fvec, 0) != 0):
                                        continue

                                    t_val = norm2_param(a, 0)
                                    p_val = norm2_param(b, 1)
                                    q_val = norm2_param(c, 0)
                                    r_val = norm2_param(d, 0)
                                    u_val = norm2_param(fvec, 0)

                                    row = (a + b + c + d + fvec +
                                           [p_val, q_val, r_val, t_val, u_val])
                                    buf[w, :] = np.array(row, dtype=np.int16)
                                    w += 1
                                    if w == chunk_rows:
                                        flush()
        flush()
    return total

def load_embeddings(path):
    """Load file created by dump_embeddings_to_file() as (n,30) int16 array."""
    arr = np.fromfile(path, dtype=np.int16)
    if arr.size % 30 != 0:
        raise ValueError("Corrupt file: length not divisible by 30.")
    return arr.reshape(-1, 30)


# ---- Timed execution ----
outfile = "embeddings_int16.bin"
start_time = time.perf_counter()
n2 = dump_embeddings_to_file(outfile, domain=range(-3,4), chunk_rows=200_000, overwrite=True)
end_time = time.perf_counter()

print("Family 2 solutions written:", n2)
print(f"Elapsed time: {end_time - start_time:.3f} seconds")

sols2_30 = load_embeddings(outfile)
print("Family 2 array shape:", sols2_30.shape)

Family 2 solutions written: 7373088
Elapsed time: 33099.060 seconds
Family 2 array shape: (7373088, 30)


In [10]:
def combine_two_families(sols1_with_tag, sols2_30):
    """
    sols1_with_tag: (N1, 31), first column is source=1.
    sols2_30      : (N2, 30), no tag yet.

    Returns:
      sols1_tagged (N1, 31)  [unchanged],
      sols2_tagged (N2, 31)  [with leading column 2],
      combined     (N1+N2,31)
    """
    sols1_tagged = sols1_with_tag

    sols2_tagged = np.hstack([
        2 * np.ones((sols2_30.shape[0], 1), dtype=np.int64),
        sols2_30.astype(np.int64)
    ])

    combined = np.vstack([sols1_tagged.astype(np.int64), sols2_tagged])
    return sols1_tagged, sols2_tagged, combined

# Example end-to-end usage:

# Family 1
sols1 = solve_system(max_results=100_000_000_000)

# Family 2
outfile = "embeddings_int16.bin"
_ = dump_embeddings_to_file(outfile, domain=range(-3,4), chunk_rows=200_000, overwrite=True)
sols2_30 = load_embeddings(outfile)

# Combine
sols1_tagged, sols2_tagged, all_solutions = combine_two_families(sols1, sols2_30)
print("Family 1 tagged shape:", sols1_tagged.shape)
print("Family 2 tagged shape:", sols2_tagged.shape)
print("Combined shape:", all_solutions.shape)
# all_solutions: rows of
#   [source, a1..a5, b1..b5, c1..c5, d1..d5, f1..f5, p, q, r, t, u]

In [ ]:
import numpy as np

def filter_unique_pqrst(sols):
    """
    Keep one representative for each distinct (p,q,r,t,u) quintuple.
    Assumes sols has columns 26–30 equal to p,q,r,t,u.
    """
    seen_pqrst = set()
    filtered = []

    for sol in sols:
        # Extract (p,q,r,t,u) from columns 26–30
        pqrst = (sol[26], sol[27], sol[28], sol[29], sol[30])

        if pqrst not in seen_pqrst:
            seen_pqrst.add(pqrst)
            filtered.append(sol)

    # Preserve original dtype
    return np.array(filtered, dtype=sols.dtype)

unique_pqrst_solutions = filter_unique_pqrst(combined)
print(len(unique_pqrst_solutions))

In [33]:
import numpy as np

def build_basis3(sol):
    """
    Construct the 6×6 lattice basis from a solution.

    Accepted formats:
      - dict:
          {
            "a": [a1..a5], "b": [b1..b5],
            "c": [c1..c5], "d": [d1..d5],
            "f": [f1..f5], "source": 1 or 2 (optional)
          }
        If "source" is missing, we default to 2.

      - array-like (length ≥ 31):
          [source, a1..a5, b1..b5, c1..c5, d1..d5, f1..f5, p, q, r, t, u]

    Output:
      6×6 int64 matrix with rows v₁,…,v₆, where
        v₁ = (1, -1, 0, 0, 0, 0)
      and the remaining rows depend on the source tag (family 1 or 2).
    """
    if isinstance(sol, dict):
        # Default to source=2 if not provided
        source = int(sol.get("source", 2))
        a = np.asarray(sol["a"], dtype=np.int64).tolist()
        b = np.asarray(sol["b"], dtype=np.int64).tolist()
        c = np.asarray(sol["c"], dtype=np.int64).tolist()
        d = np.asarray(sol["d"], dtype=np.int64).tolist()
        f = np.asarray(sol["f"], dtype=np.int64).tolist()
    else:
        # Combined-row convention: [source, a(5), b(5), c(5), d(5), f(5), p,q,r,t,u]
        vec = np.asarray(sol, dtype=np.int64).ravel()
        if vec.size < 31:
            raise ValueError(
                "Expected 31 entries: [source, a(5), b(5), c(5), d(5), f(5), p,q,r,t,u]."
            )
        source = int(vec[0])
        a = vec[1:6].tolist()
        b = vec[6:11].tolist()
        c = vec[11:16].tolist()
        d = vec[16:21].tolist()
        f = vec[21:26].tolist()

    a1,a2,a3,a4,a5 = a
    b1,b2,b3,b4,b5 = b
    c1,c2,c3,c4,c5 = c
    d1,d2,d3,d4,d5 = d
    f1,f2,f3,f4,f5 = f

    # First basis vector (common to both families)
    v1 = np.array([1, -1, 0, 0, 0, 0], dtype=np.int64)

    if source == 1:
        # Family 1 ("old" embedding)
        v2 = np.array([a1, a1,     a2,     a3, a4, a5], dtype=np.int64)
        v3 = np.array([b1, b1 + 1, b2,     b3, b4, b5], dtype=np.int64)
        v4 = np.array([c1, c1,     c2,     c3, c4, c5], dtype=np.int64)
        v5 = np.array([d1, d1,     d2,     d3, d4, d5], dtype=np.int64)
        v6 = np.array([f1, f1,     f2,     f3, f4, f5], dtype=np.int64)

    elif source == 2:
        # Family 2 ("new" parametrized embedding) matching dot_param/norm2_param
        v2 = np.array([a1, a2,       a1 + a2,     a3, a4, a5], dtype=np.int64)
        v3 = np.array([b1, b2,       b1 + b2 + 1, b3, b4, b5], dtype=np.int64)
        v4 = np.array([c1, c2,       c1 + c2,     c3, c4, c5], dtype=np.int64)
        v5 = np.array([d1, d2,       d1 + d2,     d3, d4, d5], dtype=np.int64)
        v6 = np.array([f1, f2,       f1 + f2,     f3, f4, f5], dtype=np.int64)

    else:
        raise ValueError(f"Unknown source tag {source} (expected 1 or 2).")

    return np.vstack([v1, v2, v3, v4, v5, v6])

In [34]:
def remove_wu_obstructed3(solutions):
    """
    Filter out Wu–obstructed lattices.
    For each solution, construct its 6×6 basis via build_basis3()
    and test the Wu obstruction condition.
    """
    filtered = []
    for sol in solutions:
        basis = build_basis3(sol)
        if not wu_obstruction(basis):  # retain only unobstructed cases
            filtered.append(sol)
    return filtered


wu_obstructed_solutions3 = remove_wu_obstructed3(unique_pqrst_solutions)
print(len(wu_obstructed_solutions3))

In [36]:
def filter_full_rank_lattices3(solutions):
    """
    Retain only solutions whose associated 6×6 lattice basis is full rank.
    Uses build_basis3() to construct the basis for each entry.
    """
    return [sol for sol in solutions if is_full_rank(build_basis3(sol))]


def removed_full_rank3(solutions):
    """
    Collect the degenerate (non–full-rank) cases for diagnostic use.
    """
    return [sol for sol in solutions if not is_full_rank(build_basis3(sol))]


# Apply full-rank filtering after Wu obstruction elimination
fullRankFiltered3 = filter_full_rank_lattices3(wu_obstructed_solutions3)
notFullRank3 = removed_full_rank3(wu_obstructed_solutions3)

print(f"{len(fullRankFiltered3)} out of {len(wu_obstructed_solutions3)} are full-rank.")
print(f"{len(notFullRank3)} are not full-rank.")

3098 out of 3239 are full-rank.
141 are not full-rank


In [37]:
import numpy as np

def reformat_to_dict(solutions):
    """
    Convert a (N×30) array of lattice data into a list of dictionaries.
    Each row corresponds to:
        [a1..a5, b1..b5, c1..c5, d1..d5, f1..f5, p, q, r, t, u].
    """
    arr = np.asarray(solutions)
    out = []
    for row in arr:
        row = row.tolist()
        a = list(map(int, row[0:5]))
        b = list(map(int, row[5:10]))
        c = list(map(int, row[10:15]))
        d = list(map(int, row[15:20]))
        f = list(map(int, row[20:25]))
        p, q, r, t, u = map(int, row[25:30])
        out.append({
            "a": a, "b": b, "c": c, "d": d, "f": f,
            "p": p, "q": q, "r": r, "t": t, "u": u
        })
    return out


# Reformat the full-rank set into readable dictionaries
pretty_solutions = reformat_to_dict(fullRankFiltered)

# Preview a few entries
for sol in pretty_solutions[:5]:
    print(sol)

{'a': [1, -2, -3, 0, 0], 'b': [0, -1, 1, -1, -1], 'c': [0, 0, 0, 0, 1], 'd': [-1, 0, 0, 0, 0], 'f': [1, 0, 0, 1, -1], 'p': 0, 'q': 4, 'r': 2, 't': 1, 'u': 17}
{'a': [1, -2, -3, 0, 0], 'b': [0, -1, 1, -1, -1], 'c': [0, 0, 0, 0, 1], 'd': [-1, 0, 0, 1, -1], 'f': [0, 0, 0, 0, 0], 'p': 1, 'q': 4, 'r': 2, 't': 2, 'u': 17}
{'a': [1, -2, -3, 0, 0], 'b': [0, -1, 1, -1, 0], 'c': [0, 0, 0, 1, -1], 'd': [-1, 0, 0, 0, 0], 'f': [1, 0, 0, 0, 1], 'p': 0, 'q': 3, 'r': 3, 't': 1, 'u': 17}
{'a': [1, -2, -3, 0, 0], 'b': [0, -1, 1, -1, 0], 'c': [0, 0, 0, 1, -1], 'd': [0, 0, 0, 0, 1], 'f': [-1, 0, 0, 0, 1], 'p': 1, 'q': 3, 'r': 2, 't': 2, 'u': 17}
{'a': [1, -2, -2, -1, 0], 'b': [0, 0, 0, -1, -1], 'c': [0, 0, 0, 0, 1], 'd': [-1, 0, 0, 0, 0], 'f': [1, -1, 2, 0, -1], 'p': 0, 'q': 3, 'r': 2, 't': 1, 'u': 13}


In [38]:
def canonical_weights(p, q, r, t, u, allow_center_swap=False, s_fixed=2):
    """
    Produce a canonical 5-tuple key for (p, q, r, t, u) under the
    H-plumbing weight symmetries:
      – Always: swap right leaves r ↔ u
      – Always: swap left leaves t ↔ s (with s fixed = 2)
      – Optionally: swap centers p ↔ q together with their adjacent pairs.
    """
    left  = tuple(sorted((int(t), int(s_fixed))))
    right = tuple(sorted((int(r), int(u))))
    k1 = (int(p), int(q)) + left + right
    if not allow_center_swap:
        return k1
    k2 = (int(q), int(p)) + right + left
    return k1 if k1 <= k2 else k2


def filter_solutions_by_weights(pretty_solutions, allow_center_swap=False, s_fixed=2):
    """
    Deduplicate solutions modulo the weight symmetries defined in canonical_weights().
    """
    seen = set()
    out = []
    for sol in pretty_solutions:
        key = canonical_weights(sol["p"], sol["q"], sol["r"], sol["t"], sol["u"],
                                allow_center_swap=allow_center_swap, s_fixed=s_fixed)
        if key not in seen:
            seen.add(key)
            out.append(sol)
    return out

# Identify distinct solutions up to weight symmetries
unique_pretty_solutions = filter_solutions_by_weights(pretty_solutions, allow_center_swap=True)
print(len(unique_pretty_solutions))

2549

In [40]:
from numba import njit
import numpy as np

@njit
def generate_unit_cube(dim):
    """
    Return all 2^dim corners of the unit cube [0,1]^dim as rows of an array.
    """
    num_corners = 1 << dim
    cube = np.zeros((num_corners, dim), dtype=np.float64)
    for i in range(num_corners):
        for j in range(dim):
            cube[i, j] = (i >> j) & 1
    return cube


@njit
def is_cubiquitous_fast(basis, bound=3, rtol=1e-12):
    """
    Numerical cubiquity test for a lattice with basis rows v_1,…,v_6.

    Checks that every unit cube in the region [-bound, bound]^6 has at least
    one lattice point. Uses an SVD-based pseudoinverse of A = basis^T and
    rejects numerically rank-deficient matrices.
    """
    # Work with columns as basis vectors: A c = point
    A = basis.T.astype(np.float64)

    # One global SVD
    U, S, Vh = np.linalg.svd(A)
    if S.size == 0:
        return False

    # Numerical rank from relative cutoff
    cutoff = S[0] * rtol
    invS = np.zeros_like(S)
    rank = 0
    for i in range(S.shape[0]):
        if S[i] > cutoff:
            invS[i] = 1.0 / S[i]
            rank += 1

    # If rank < n, the lattice is degenerate -> not cubiquitous
    if rank < A.shape[0]:
        return False

    # Precompute unit cube corners in R^6
    unit_cube = generate_unit_cube(A.shape[0])

    # Tolerance scaled by condition number and operator norm
    cond = S[0] / S[-1]
    scale = max(np.linalg.norm(A), 1.0)
    tol = cond * np.finfo(np.float64).eps * scale * 10.0

    def svd_apply_pinv(b):
        """
        Apply the pseudoinverse A^+ via SVD:
            A^+ b = V^T diag(1/S) U^T b.
        """
        y = U.T @ b
        for i in range(y.shape[0]):
            y[i] *= invS[i]
        return Vh.T @ y

    # For each integer translate of the cube, check if at least
    # one corner lies on the lattice (up to tolerance).
    rng = range(-bound, bound + 1)
    for i0 in rng:
        for i1 in rng:
            for i2 in rng:
                for i3 in rng:
                    for i4 in rng:
                        for i5 in rng:
                            coord = np.array([i0, i1, i2, i3, i4, i5], dtype=np.float64)
                            cube_passed = False
                            for corner in unit_cube:
                                point = corner + coord
                                coeffs = svd_apply_pinv(point)
                                rounded = np.rint(coeffs)
                                recon = A @ rounded

                                ok = True
                                for k in range(recon.shape[0]):
                                    if abs(recon[k] - point[k]) >= tol:
                                        ok = False
                                        break
                                if ok:
                                    cube_passed = True
                                    break
                            if not cube_passed:
                                return False
    return True


cubiquitous_solutions = []
for sol in unique_pretty_solutions:
    # build_basis3 returns 6×6 basis with rows v_1,…,v_6
    B = build_basis3(sol).astype(np.float64)
    if is_cubiquitous_fast(B, bound=2):
        cubiquitous_solutions.append(sol)
        print("Cubiquitous:", sol)

print("Number of cubiquitous solutions:", len(cubiquitous_solutions))

Number of cubiquitous solutions: 1450


In [51]:
from collections import defaultdict

# ---- Symmetry on weights only (canonical key) ----

def canonical_weights(p, q, r, t, u, *, allow_center_swap=True, s_fixed=2):
    """
    Canonical key for the weight quintuple with one fixed leaf s = s_fixed.
    Symmetries:
      - always: swap right leaves r ↔ u;
      - always: swap left leaves t ↔ s_fixed;
      - optionally: swap centers p ↔ q together with exchanging the two leaf pairs.
    """
    left  = tuple(sorted((int(t), int(s_fixed))))
    right = tuple(sorted((int(r), int(u))))
    k1 = (int(p), int(q)) + left + right
    if not allow_center_swap:
        return k1
    # also allow swapping the two central weights and their leaf pairs
    k2 = (int(q), int(p)) + right + left
    return k1 if k1 <= k2 else k2


# ---- Generate all H-plumbing images of a 6-tuple (p,q,r,s,t,u) ----

def h_images_6tuple(p, q, r, s, t, u):
    """
    Generate all 8 images of (p,q,r,s,t,u) under:
      - center swap (p,q) and interchange of leaf pairs (r,s) ↔ (t,u);
      - left-pair swap s ↔ t;
      - right-pair swap r ↔ u.
    """
    for center_swap in (0, 1):
        if center_swap:
            p2, q2 = q, p
            # swap leaf pairs (r,s) and (t,u)
            r2, s2, t2, u2 = t, u, r, s
        else:
            p2, q2 = p, q
            r2, s2, t2, u2 = r, s, t, u
        for swap_left in (0, 1):
            for swap_right in (0, 1):
                rr, ss, tt, uu = r2, s2, t2, u2
                if swap_left:
                    ss, tt = tt, ss
                if swap_right:
                    rr, uu = uu, rr
                yield (p2, q2, rr, ss, tt, uu)


# ---- Map a 6-tuple to canonical keys with s fixed to 2 ----

def keys_from_6tuple_with_s2(p, q, r, s, t, u):
    """
    From (p,q,r,s,t,u), collect all canonical keys that arise from
    H-plumbing symmetries and place the distinguished leaf s_fixed = 2
    on the left pair.
    """
    keys = set()
    for (pp, qq, rr, ss, tt, uu) in h_images_6tuple(p, q, r, s, t, u):
        # Left pair is (ss, tt). Exactly one of them should be 2.
        if ss == 2:
            t_other = tt
        elif tt == 2:
            t_other = ss
        else:
            continue
        key = canonical_weights(pp, qq, rr, t_other, uu,
                                allow_center_swap=True, s_fixed=2)
        keys.add(key)
    return keys


# ---- Build key -> source mapping for the filtered 6-tuples ----

def index_filtered_tuples(filtered_tuples2):
    """
    Index filtered 6-tuples (p,q,r,s,t,u,det) by all compatible
    canonical keys with s_fixed = 2.
    Returns: dict key -> list of 7-tuples.
    """
    bucket = defaultdict(list)
    for tup in filtered_tuples2:
        p, q, r, s, t, u, det = tup
        keys = keys_from_6tuple_with_s2(p, q, r, s, t, u)
        for key in keys:
            bucket[key].append(tup)
    return bucket


# ---- Build key -> source mapping for the cubiquitous solutions ----

def index_cubiquitous_solutions(cubiquitous_solutions):
    """
    Index cubiquitous solutions (with s = 2 fixed) by their canonical key.
    Each solution is a dict with entries p,q,r,t,u.
    Returns: dict key -> list of solution dicts.
    """
    bucket = defaultdict(list)
    for sol in cubiquitous_solutions:
        p = int(sol["p"]); q = int(sol["q"])
        r = int(sol["r"]); t = int(sol["t"]); u = int(sol["u"])
        key = canonical_weights(p, q, r, t, u,
                                allow_center_swap=True, s_fixed=2)
        bucket[key].append(sol)
    return bucket


# ---- Match the two sets modulo symmetry ----

def match_cubiquitous_vs_filtered(filtered_tuples2, cubiquitous_solutions):
    """
    Compare the combinatorial 6-tuples (filtered_tuples2) with numerically
    cubiquitous embeddings (cubiquitous_solutions) modulo the H-plumbing
    symmetries and the identification s = 2.

    Returns a dict with:
      - 'matched': list of blocks
          { 'key': canonical key,
            'filtered': [6-tuples with that key],
            'cubiquitous': [solutions with that key] }
      - 'unmatched_filtered': 6-tuples whose symmetry orbit has no cubiquitous match
      - 'unmatched_cubiquitous': cubiquitous solutions without a matching 6-tuple
    """
    idx_filtered = index_filtered_tuples(filtered_tuples2)
    idx_cubi     = index_cubiquitous_solutions(cubiquitous_solutions)

    keys_filtered = set(idx_filtered.keys())
    keys_cubi     = set(idx_cubi.keys())

    matched_keys = keys_filtered & keys_cubi

    matched = []
    for k in sorted(matched_keys):
        matched.append({
            "key": k,
            "filtered": idx_filtered[k],
            "cubiquitous": idx_cubi[k],
        })

    # 6-tuples whose entire symmetry orbit misses the cubiquitous keys
    unmatched_filtered = []
    for tup in filtered_tuples2:
        p, q, r, s, t, u, det = tup
        keys = keys_from_6tuple_with_s2(p, q, r, s, t, u)
        if not any((k in idx_cubi) for k in keys):
            unmatched_filtered.append(tup)

    # Cubiquitous solutions with no corresponding filtered 6-tuple
    unmatched_cubiquitous = []
    for sol in cubiquitous_solutions:
        key = canonical_weights(int(sol["p"]), int(sol["q"]),
                                int(sol["r"]), int(sol["t"]), int(sol["u"]),
                                allow_center_swap=True, s_fixed=2)
        if key not in idx_filtered:
            unmatched_cubiquitous.append(sol)

    return {
        "matched": matched,
        "unmatched_filtered": unmatched_filtered,
        "unmatched_cubiquitous": unmatched_cubiquitous,
    }


# -------------------------

out = match_cubiquitous_vs_filtered(filtered_tuples2, cubiquitous_solutions)
print("Matches:", len(out["matched"]))
for block in out["matched"][:5]:
    print("key:", block["key"])
    print("  filtered:", block["filtered"])
    print("  cubiquitous:", block["cubiquitous"])

Matches: 0


In [54]:
def build_basis_picture():
    v1 = [-1,  1,  1,  1, 1, 0]
    v2 = [1, -1,  -1, 1, 0, 1]
    v3 = [1, 1,  -1,  0, 0, 0]
    v4 = [-1,  0,  0, 0, -1, -1]
    v5 = [1, -1,  0,  1, 1, -1]
    v6 = [1,  0,  1, 1, -1, 1]
    return np.array([v1, v2, v3, v4, v5, v6], dtype=np.float64)  # keep float64

B = build_basis_picture()

# quick sanity
print("rank(B):", np.linalg.matrix_rank(B))
print("abs(det(B)):", int(round(abs(np.linalg.det(B)))))

# use your existing checker
print("cubiquitous (bound=2):", is_cubiquitous_fast(B, bound=2))
# increase search window if desired
#print("cubiquitous (bound=3):", is_cubiquitous_fast(B, bound=3))

rank(B): 6
abs(det(B)): 40
cubiquitous (bound=2): False
cubiquitous (bound=3): False
